In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense


# Load the dataset
data = pd.read_csv('/kaggle/input/births2015/Births2015.csv')

# Clean and preprocess the data
births_data['date'] = pd.to_datetime(births_data['date'])
births_data = births_data[['date', 'births']]
births_data = births_data.sort_values('date').reset_index(drop=True)

In [ ]:
# Normalize the 'births' column
scaler = MinMaxScaler(feature_range=(0, 1))
births_data['normalized_births'] = scaler.fit_transform(births_data[['births']])

# Create sequences for the RNN
def create_sequences(data, sequence_length):
    sequences = []
    targets = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        targets.append(data[i + sequence_length])
    return np.array(sequences), np.array(targets)

# Define sequence length and prepare data
sequence_length = 7
sequences, targets = create_sequences(
    births_data['normalized_births'].values, sequence_length
)

# Split the data into training and test sets
split_index = int(0.8 * len(sequences))
X_train, X_test = sequences[:split_index], sequences[split_index:]
y_train, y_test = targets[:split_index], targets[split_index:]

# Reshape the data to fit the RNN input format (samples, time_steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))



In [ ]:
# Define the RNN model
model = Sequential([
    Input(shape=(sequence_length, 1)),
    SimpleRNN(50, activation='tanh'),
    Dense(1)
])

# Compile
model.compile(optimizer='adam', loss='mse')

# Train
history = model.fit(
    X_train, y_train,
    epochs=50, batch_size=16,
    validation_data=(X_test, y_test),
    verbose=1
)

# Evaluate
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {test_loss}")

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.3512 - val_loss: 0.0621
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0329 - val_loss: 0.0496
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0173 - val_loss: 0.0388
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0114 - val_loss: 0.0362
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0086 - val_loss: 0.0379
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0093 - val_loss: 0.0350
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0082 - val_loss: 0.0334
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0082 - val_loss: 0.0334
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0099 - val_loss: 0.0324
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0112 - val_loss: 0.0364
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0123 - val_loss: 0.0373
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0074 - val_l

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Calculate accuracye
error_margin = 0.10  # 10%
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_rescaled = scaler.inverse_transform(predictions)

# Compute accuracy
correct_predictions = np.abs(predictions_rescaled - y_test_rescaled) <= (error_margin * y_test_rescaled)
accuracy = np.mean(correct_predictions) * 100
print(f"Accuracy: {accuracy:.2f}%")


# Rescale predictions back to original scale
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Accuracy: 86.11%
